# Work Log and Journal

Add new entries to the top; read in reverse chronological order

Entry format: 

**DD Month YY - short title**

Short description of work

Merges, squashes, etc. 

## How to get started again after a while off

- Start virtual env: `source ./venv_activate`
- Launch application from project root with: `python -m paperpi`
- Start uvicorn web interface: `python -m uvicorn paperpi.web.app:app --reload --host 0.0.0.0`

### TO DO:

Awesome—this is a perfect moment to set the foundation so “add another config” is mostly wiring, not new code. Here’s a clear, reusable approach that fits your “main + plugins” shape and scales.

1) Normalize the data model
- One application config file on disk:
  main:
    log_level: WARNING
    screen_mode: "1"
    ...
  plugins:
    - id: clock1
      type: clock
      enabled: true
      refresh: 30
      ...
    - id: weather_sf
      type: weather
      enabled: true
      units: metric
      ...
- Every plugin instance has:
  - a stable id (string, unique)
  - a type (plugin kind)
  - its own key/values per its plugin schema

2) Make schemas composable and discoverable
- Keep one application schema with two top-level sections:
  - main (flat field rules)
  - plugins (rules for per-instance “envelope”: id, type, enabled, plus a schema_provider token to pull per-plugin rules)
- Each plugin type provides its own schema (e.g., ${PLUGIN_SCHEMA:clock}).
- Your existing token expansion registry can support parameterized tokens:
  - ${DISPLAY_TYPES} (already)
  - ${PLUGIN_SCHEMA:clock} → returns the field rules for a clock plugin
- Providers live in paperpi/providers/… (e.g., get_plugin_schema(name)) and the daemon registers them at startup.

3) Design a generic daemon API
Keep it resource-oriented and reusable. These 6 endpoints cover everything:

Schemas
- GET /schema/app → expanded, effective app schema (with main and plugins sections resolved)
- GET /schema/plugin/{type} → expanded schema for a plugin type (resolver uses providers)

Config read/write
- GET /config/app → returns { main: {...}, plugins: [...] }
- POST /config/check/app → { config } → { problems, submitted } (validate entire app config)
- POST /config/write/app → { config } → { written, diff, message }

Plugin instance management (optional, but pays off)
- POST /config/plugin/{id}/check → { config } → validate just that instance (daemon looks up type → schema)
- POST /config/plugin/{id}/write → write only that instance back into plugins[] (deep-merge by id)
- POST /config/plugin → { id, type, config } add new instance
- DELETE /config/plugin/{id} → remove instance
- POST /config/plugin/{id}/reorder → move instance up/down (optional)

Internally, the daemon always validates on the effective schema (tokens already expanded once at startup as you prefer).

4) Web layer: one generic UI for any section
- Keep one config.html template and move the input widgets into a Jinja macro: _field.html::render_field(key, value, rules, result)
- Build two renderers in the route:
  - For main: iterate key/value pairs from config["main"]
  - For plugins: iterate the list; for each instance, fetch the plugin’s effective schema (/schema/plugin/{type}) and render fields using the macro
- Keep the existing schema-aware coercion (coerce_by_schema) and apply it:
  - To main fields using app_schema["main"]
  - To each plugin instance using the per-plugin schema returned

5) Validation flow (generic, section-aware)
- On Check:
  1) Load app_schema (and per-plugin schemas on demand)
  2) Coerce submitted values per schema(s)
  3) POST to POST /config/check/app
  4) Show problems inline; compute changed (vs current) to enable the write button
- On Write:
  - Post the already-coerced, already-validated payload forward (payload_json) to POST /config/write/app
  - Don’t re-fetch schema, don’t re-validate

6) Reusable utilities (you already have many)
- coerce_by_schema(values, schema) → works for both main and plugin instance dicts
- update_yaml_file(path, changes) → write-by-diff
- diff_configs(old, new) → shallow compare (extend to nested for plugins if you want finer “changed” detection)
- Token expansion registry → now add a parameterized provider PLUGIN_SCHEMA:<type>

7) Keep back-compat while migrating
- Alias current routes:
  - GET /config → GET /config/app
  - POST /config/check_app → POST /config/check/app
  - POST /config/write_app → POST /config/write/app
- Update the template form action/formaction to the new endpoints when you’re ready

8) UX suggestions (small but helpful)
- After a successful Check (no problems), display a compact list of what changed (e.g., “3 fields changed: log_level, refresh_interval, splash”).
- In plugins list, show per-instance headers like “Clock (id=clock1)” with enable/disable and a “Remove” button (wired later).
- “Add plugin” flow:
  - Choose type → daemon returns default config from GET /schema/plugin/{type} (or a token-provided default) → append to list with a temp id until written.

9) A minimal step-by-step to get there
1) Daemon
   - Add /schema/app and /schema/plugin/{type} that return effective schemas (expanded once).
   - Add /config/check/app and /config/write/app (you already have check+write split; just generalize names).
2) Web
   - Introduce GET /config/app (keep GET /config as alias).
   - Switch your existing POST handlers to {name} forms (/config/check/app, /config/write/app) and keep the alias routes temporarily.
   - Move field rendering into a macro so you can reuse for plugins.
3) Plugins UI
   - Render the list of instances; for each, fetch plugin schema and render fields inside a \<fieldset\>.
   - Submit the entire app payload in one shot (main + plugins), so the daemon validates atomically.

10) Why this works (reusability wins)
- One template, one coercer, one validator, one writer → parameterized by name or plugin type.
- New plugin type? Add a schema provider + template rendering piggybacks on the macro. No new routes.
- New config section? Add to the registry + schema and the same UI/controller logic handles it.

If you want, I can turn just your current “app” routes into the {name} version first (keeping old paths working), and sketch the plugin rendering macro—then you’ll have a pattern to extend to other sections without duplicating code.

#### `/config/` route

* [x] display_type accepts only valid values
- [x] display types are pulled from epdlib
- [x] where did the true/false boxes go for unchecked?
- [x] should use daemon schema route rather than trying to load files its self

## 14 August 2025

Plan: pull display_type from epdlib and populate schema with this

- use epdlib to populate schema with allowed display types
- fix type coersion
- add write to 


## 13 August 2025

* fix issues in relative imports in `__init__`, `__main__` and `paperpi.py`
* create `config_routes.handle_check_config` to validate submitted dictionaries that include schema and configurations
*  Add do_POST method to `http_handler` to accept JSON input
* add daemon route to show available schema files at `/schema/` and contents of schema files by name at `/schema/schema_name`
* update web route `config/` submission to get config from daemon/schema and check values, highlight problem areas
* update form submission html to make sure that false/empty form values are always included





## 1 May

- Refactor of daemon
    - [x] return status
    - [x] return config (dynamic)
    - [ ] return schema (dynamic?)
    - [ ] validate submitted config

- Web Front End
    - [ ] use new config and schema api to show configuration
    - [ ] use new validate routines
    - [ ] 

## 30 April

### WIP: br web_config
- [x] Develop web pages that allow editing using the values from the YAML schema
- [x] Total refactor of daemon.py - it's a mess
    - [ ] work on loading and reloading of daemon data
- [ ] Refactor API to deliver config/schema and schema/app and schema/basic_plugin
- [ ] Refactor config.py and config.html to use the new schema deliver systems
- [ ] Validate submitted values against schema
    - [ ] reject invalid config & highlight any sections that are incorrect
    - [ ] save when good

## 29 April

### WIP: br web_config

Create FastAPI interface for editing configuration files and validating before saving

- [x] Configuration validates against known screen types dynamically based on `epdlib.Screen.list_compatible`
- [ ] Develop web pages that allow editing using the values from the YAML schema
- [ ] Validate submitted values against schema
    - [ ] reject invalid config & highlight any sections that are incorrect
    - [ ] save when good

## 27 April

- WIP: make sure that PluginManager class is capable of returning necessary JSON data for plugins
    - [x] Updated docstrings, added example code
    - [ ] Need to tidy up test code and commit changes
- WIP: configure plugin manager in `daemon.py`
    - [x] Build routes into API for displaying plugin information -- need to make sure it is all json safe
    - [x] Add basic PluginManager configuration in daemon.py
    - [x] Add static resolution and color modes for testing
    - [x] Load and activate plugins

- Started to work with epdlib.Screen to list compatible modules and try to display an image on screen. Something is broken with the import 

## 26 Apirl

- moved all loading of configuration information into daemon; this allows a `/reload` route that allows reloading the configuration files when changes are made.
- update daemon API to add routes for viewing configuration
- consolodate web interface configuration page into single page for viewing/editing configuration
- create settings object for fastapi server for managing settings across routes


## 23 April '25

- shifted focus to creating an HTTP socket run by the daemon for accessing program state, configuration, etc.
    - created new branch daemon_api
- Daemon launches with running loop and stays active until paperpi stops; serves current configuration
- Add command to shutdown
- Restructured into ./paperpi/ sub directory
- Add daemon methods for returning json formatted configuration
- Add web pages for displaying configuration and editing configuration (not yet connected)

## 22 April '25

- refactored everything to be more modular and merged a `refactor` branch in
- created `./web/` structure and moved fastapi app into structure
- Goal: create a web app that can be launched from the main application and receives data from it and can interface with it.
    - short term: show configuration on web interface screen
    - start/stop uvicorn using a daemon process that runs externally from the main process (?)
- Add startup of web app from within main: There's something wrong with the argv that are being processed, however.

### Next Time...

- [ ] Get the daemon to start and start the web service
- [ ] Get the web service to show the current configuration
    - [ ] Allow editing of configuration from web
- [ ] Add appropriate waveshare deps
- [ ] Read configuration and configure display
- [ ] Cycle plugins to screen
- [ ] Try activating and deactivating plugins while running (not quite ready for this?)
- [ ] 

## 21 April '25

- [x] Connect low-resolution display
- [x] Add waveshare libraries

### Next Time...

- [ ] Add appropriate waveshare deps
- [ ] Read configuration and configure display
- [ ] Cycle plugins to screen
- [ ] Try activating and deactivating plugins while running (not quite ready for this?)
- [ ] 

## 13 Apirl '25 - refactor_config

- [x] Move all the reading, writing and validation functions in to `library.config_utils`.
- [x] successfully run main and load module

### Next Time...

- [ ] Connect low-resolution display
- [ ] Add appropriate waveshare deps
- [ ] Read configuration and configure display
- [ ] Cycle plugins to screen
- [ ] Try activating and deactivating plugins while running (not quite ready for this?)
- [ ] 

## 13 April '25 - Getting back up to speed

Figuring out how the pieces fit together. 

### paperpi.py 

Main program, loads & validates configuration

### paperpi_daemon.py

Handles daemon related tasks, sets pid, restarts/stops (???)

### paperpi_web.py

??? - Leftovers from testing?

### wip-plugin_manger.py

??? Leftovers from testing - looks like I moved this into `./library/plugin_manager.py`. It likely should be removed.

### ./library/base_plugin.py

Parent class for plugins 

### ./library/exceptions.py

Exception classes

### ./library/plugin_manager.py

Class that manages life cycle of plugins


## NEXT STEPS:

### Basic execution

Try to get `paperpi.main` to execute and see where we are.

`paperpi.load_validate_config` looks like it should return a dictionary of all the errors it encountered organized by type. Right now it returns a list of errors. Also, there seems to be a problem with reading hte configuration overall.

`PluginManager` confusingly validates configurations; move this into a library of some kind and then use in the plugin manager.